In [1]:
# Data manipulation
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Linear Algebra
import cvxopt as opt
from cvxopt import solvers, blas

# Turn off progress printing
solvers.options['show_progress'] = False

# Data visualization
import matplotlib.pyplot as plt

# Type hinting
from typing import Tuple

In [2]:
DATA_PATH = 'data.csv'
WORK_DAYS_YEAR = 252
# TODO: Get the risk-free rate from a reliable source
RISK_FREE_RATE = 0.038  # 10-year US Treasury bond yield 3.8% annualized estimate
NUM_PORTFOLIOS = 10_000

In [3]:
def get_data() -> pd.DataFrame:
    data = pd.read_csv(DATA_PATH, parse_dates=['Date'])
    data['Return'] = data.groupby('Ticker')['PX_LAST'].pct_change()
    data.dropna(inplace=True)
    
    return data


def annualize_return(returns_df: pd.DataFrame) -> pd.DataFrame:
    return returns_df.mean() * WORK_DAYS_YEAR


def annualize_cov(returns_df: pd.DataFrame) -> pd.DataFrame:
    return returns_df.cov() * WORK_DAYS_YEAR

In [4]:
def portfolio_performance(weights: np.ndarray, returns_array: np.ndarray, cov_matrix: np.ndarray) -> Tuple[float, float]:
    '''
    Calculate the expected portfolio return and volatility.
    
    This function computes the expected return and volatility (standard deviation) of a portfolio
    given the asset weights, expected returns, and the covariance matrix of the asset returns.
    
    Parameters:
        weights (np.ndarray): Array of assets weights in the portfolio.
        returns_array (np.ndarray): Array of expected returns for each asset.
        cov_matrix (np.ndarray): Covariance matrix of asset returns.
        
    Returns:
        Tuple[float, float]: A tuple containing the portfolio's expected return and volatility.
    '''
    port_ret = np.dot(weights, returns_array)
    port_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    
    return port_ret, port_vol

In [5]:
def mvp(cov_matrix: np.ndarray, annual_returns: np.ndarray, *, short: bool = False) -> np.ndarray:
    '''
    Calculate the weights of the Minimum Variance Portfolio (MVP).
    
    The MVP is the portfolio with the lowest possible variance (risk) for a given set of assets.
    This function uses quadratic programming to find the optimal weights that minimize the portfolio variance.
    
    Parameters:
        cov_matrix (np.ndarray): Covariance matrix of asset returns.
        annual_returns (np.ndarray): Expected annual returns of each of the assets.
        short (bool, optional, keyword-only, default=False): If True, allows short selling (weights can be negative).
    
    Returns:
        np.ndarray: Optimal weights for the Minimum Variance Portfolio.
    '''
    short_tuple = (-1, 1) if short else (0, 1)  # Each weight must be between 0 and 1, or -1 and 1 for shorting
    
    num_assets = len(cov_matrix)
    
    # TODO: Review the "args" variable below
    args = (cov_matrix, )
    
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})  # Sum of weights must be 1
    bounds = tuple(short_tuple for asset in range(num_assets))
    initial_weights = num_assets * [1. / num_assets, ]  # Equally weighted portfolio as starting point
    
    result = minimize(
        lambda w: portfolio_performance(w, annual_returns, cov_matrix)[1],  # Minimize volatility
        initial_weights,
        # args=args,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    
    return result.x

In [6]:
def efficient_frontier(returns_array: np.ndarray, cov_matrix: np.ndarray, num_portfolios: int = NUM_PORTFOLIOS) -> np.ndarray:
    '''
    Calculate the efficient frontier for a given set of asset returns and covariance matrix.
    
    The efficient frontier represents a set of optimal portfolios that offer the highest expected return
    for a defined level of risk or the lowest risk for a given level of expected return.
    
    Parameters:
        returns_array (np.ndarray): Array of expected returns for each asset.
        cov_matrix (np.ndarray): Covariance matrix of asset returns.
        num_portfolios (int, optional, default=NUM_PORTFOLIOS): Number of portfolios to simulate.
    
    Returns:
        np.ndarray: A 2D array where each column represents a portfolio with the following rows:
                    - Expected return
                    - Volatility (standard deviation)
                    - Sharpe ratio
    '''
    results = np.zeros((3, num_portfolios))
    
    for i in range(num_portfolios):
        weights = np.random.random(len(returns_array))
        weights /= np.sum(weights)
        
        portfolio_return, portfolio_volatility = portfolio_performance(weights, returns_array, cov_matrix)
        
        results[0, i] = portfolio_return
        results[1, i] = portfolio_volatility
        results[2, i] = (portfolio_return - RISK_FREE_RATE) / portfolio_volatility  # Sharpe ratio
        
    return results

In [7]:
def optimal_portfolio(results: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    '''
    Calculate the optimal portfolio and the efficient frontier.

    Parameters:
        results (np.ndarray): Array of portfolio results (expected returns, volatility, Sharpe ratio).

    Returns:
        tuple(np.ndarray, np.ndarray, np.ndarray): Optimal weights, results, and risks of the efficient frontier.
    '''
    n = len(results)
    results = np.asmatrix(results)
    
    N = 100
    mus = [10**(5.0 * t/N - 1.0) for t in range(N)]
    
    # Convert to cvxopt matrices
    S = opt.matrix(np.cov(results))
    pbar = opt.matrix(np.mean(results, axis=1))
    
    # Create constraint matrices
    G = -opt.matrix(np.eye(n))  # negative n x n identity matrix
    h = opt.matrix(0.0, (n ,1))  # n x 1 zero matrix
    A = opt.matrix(1.0, (1, n))  # 1 x n matrix of ones
    b = opt.matrix(1.0)  # 1 x 1 matrix of ones
    
    # Calculate efficient frontier weights using quadratic programming
    portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x'] for mu in mus]
    
    # Calculate risks and results for frontier
    results = [blas.dot(pbar, x) for x in portfolios]  # Expected returns
    risks = [np.sqrt(blas.dot(x, S*x)) for x in portfolios]  # Volatility
    
    # Calculate the 2nd degree polynomial of the frontier curve
    m1 = np.polyfit(results, risks, 2)  # Polynomial coefficients
    
    # TODO: Review the following code
    # Probably the problem is related to the FIRST daily return of the portfolio always being 0
    if m1[2] < 0:
        raise ValueError('The polynomial coefficient is negative')
        
    x1 = np.sqrt(m1[2] / m1[0])  # Maximum Sharpe ratio
    
    # Calculate the optimal portfolio
    P = opt.matrix(x1 * S)  # Maximum Sharpe ratio * covariance matrix
    print(P)
    wt = solvers.qp(P, -pbar, G, h, A, b)['x']
    
    return np.asarray(wt), results, risks

In [8]:
def plot_efficient_frontier(results: np.ndarray, *, short: bool = True) -> None:
    '''
    Plot the efficient frontier.
    
    This function plots the efficient frontier based on the provided results.
    It distinguishes between portfolios that allow short selling and those that
    do not by using different line styles.
    
    Parameters:
        results (np.ndarray): A 2D array where each column represents a portfolio with the following rows:
                              - Expected return
                              - Volatility (standard deviation)
                              - Sharpe ratio
        short (bool, optional, keyword-only, default=True): If True, indicates that short selling is allowed and the frontier will be extended with a dotted line.
    '''
    # Filter out invalid portfolios (zero or infinite values)
    valid_mask = np.all(np.isfinite(results) & (results > 0), axis=0)
    filtered_results = results[:, valid_mask]
    
    if filtered_results.shape[1] == 0:
        raise ValueError('No valid portfolios to plot. All input results are invalid (zero or infinite values).')
    
    _, frontier_returns, frontier_risks = optimal_portfolio(results)
    
    plt.figure(figsize=(10, 6))
    plt.plot(frontier_risks, frontier_returns, 'r-', label='Efficient Frontier')  # Efficient frontier
    plt.scatter(results[1, :], results[0, :], c=results[2, :], cmap='YlGnBu', marker='o')  # Portfolios results
    
    if short:
        # Extend the frontier with a dotted line for short selling
        extended_returns = np.concatenate((frontier_returns, frontier_returns[::-1]))
        extended_risks = np.concatenate((frontier_risks, frontier_risks[::-1]))
        plt.plot(extended_risks, extended_returns, 'r--', label='Efficient Frontier (Short Selling)')
    
    plt.title('Mean-Variance Efficient Frontier')
    plt.xlabel('Risk/Volatility (Standard Deviation)')
    plt.ylabel('Expected Return')
    plt.colorbar(label='Sharpe Ratio')
    plt.show()

In [9]:
df: pd.DataFrame = get_data()
df.head()

/tmp/ipykernel_91325/2511570019.py:3: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  data['Return'] = data.groupby('Ticker')['PX_LAST'].pct_change()


,Date,PX_LAST,CUR_MKT_CAP,PX_VOLUME,Ticker,Market_Value,Return
1,2014-01-03,76.11,133746.5552,4061042.0,DIS,1.017945e+07,-0.002098
2,2014-01-06,75.82,133236.9441,6816693.0,DIS,1.010203e+07,-0.003810
3,2014-01-07,76.34,134150.7295,4511157.0,DIS,1.024107e+07,0.006858
4,2014-01-08,75.22,132182.5763,10914858.0,DIS,9.942773e+06,-0.014671
5,2014-01-09,74.90,131620.2468,8077726.0,DIS,9.858356e+06,-0.004254


In [10]:
returns: pd.DataFrame = df.pivot(index='Date', columns='Ticker', values='Return')
returns

Ticker,AKAM,ALL,AMD,APH,BK,BSX,CMG,DIS,DRI,DVN,...,SBAC,SJM,SMCI,STT,TXT,UAL,UHS,VLO,VRTX,WY
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-03,-0.001719,-0.004108,0.012658,0.003647,0.011574,0.001679,0.014997,-0.002098,-0.005814,-0.002126,...,-0.006899,-0.005097,0.016423,0.021050,0.000551,0.058839,0.011039,-0.010427,-0.005147,0.002239
2014-01-06,-0.007320,-0.003938,0.032500,-0.009194,-0.000286,0.033529,-0.007529,-0.003810,-0.018864,-0.012781,...,0.000114,-0.006009,-0.007181,-0.000134,-0.004131,-0.014768,-0.012268,0.009726,0.027910,-0.010211
2014-01-07,0.019519,0.004330,0.012107,0.006758,-0.011731,0.008921,0.010905,0.006858,0.010960,0.020415,...,0.008426,0.006145,0.010850,0.007364,-0.011062,-0.017022,0.028692,0.031908,-0.013377,0.002901
2014-01-08,-0.007020,0.003187,0.000000,0.005583,0.003474,0.028135,0.005253,-0.014671,-0.018449,-0.002765,...,0.005420,-0.032998,-0.001789,-0.002525,0.000559,0.060222,0.002656,0.008557,0.009934,-0.009322
2014-01-09,0.020137,0.005792,-0.021531,0.012336,0.001154,-0.000782,0.009704,-0.004254,-0.005230,-0.008645,...,-0.013477,-0.001935,-0.009558,0.003997,0.001118,0.067772,0.016859,0.020054,0.014489,0.005840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,0.001088,0.014771,-0.002216,0.007325,0.001362,0.004492,-0.001192,-0.010867,-0.002281,-0.000872,...,-0.001509,0.008895,-0.047538,0.002337,0.004496,-0.000705,0.013708,0.002122,0.011532,0.007329
2023-12-26,-0.001170,0.004949,0.027292,0.004848,0.009714,0.001610,0.006575,-0.000769,0.013285,0.018555,...,0.004614,0.004248,0.015912,0.006866,0.003730,-0.011046,0.006859,0.007489,0.005947,0.007858
2023-12-27,-0.004017,0.002245,0.018548,-0.000201,0.001924,0.000536,-0.005195,-0.006267,-0.002683,-0.005572,...,0.006414,-0.001437,0.002276,0.003217,-0.000248,-0.008317,-0.009862,-0.010286,0.000957,0.003754


In [11]:
annualized_returns: pd.DataFrame = annualize_return(returns)
annualized_returns.head()

Ticker
AKAM    0.142589
ALL     0.124364
AMD     0.528008
APH     0.178581
BK      0.077880
dtype: float64

In [12]:
cov_matrix: pd.DataFrame = annualize_cov(returns)
cov_matrix.head()

Ticker,AKAM,ALL,AMD,APH,BK,BSX,CMG,DIS,DRI,DVN,...,SBAC,SJM,SMCI,STT,TXT,UAL,UHS,VLO,VRTX,WY
Ticker,,,,,,,,,,,,,,,,,,,,,
AKAM,0.098273,0.019403,0.047715,0.029772,0.025343,0.025641,0.024300,0.025215,0.017946,0.031629,...,0.028822,0.015473,0.032948,0.029238,0.030415,0.026799,0.022492,0.023048,0.035895,0.027389
ALL,0.019403,0.055599,0.026929,0.027809,0.034641,0.026836,0.020248,0.026360,0.035434,0.044202,...,0.020518,0.014508,0.027055,0.040906,0.041015,0.043891,0.033958,0.039457,0.019793,0.035479
AMD,0.047715,0.026929,0.337691,0.056073,0.043463,0.043654,0.056221,0.045108,0.045303,0.070558,...,0.038507,0.010414,0.080263,0.058883,0.054445,0.057232,0.038900,0.047831,0.050958,0.055564
APH,0.029772,0.027809,0.056073,0.054525,0.036350,0.033387,0.030875,0.032846,0.039939,0.050451,...,0.026347,0.011133,0.046458,0.045504,0.048591,0.053967,0.037132,0.040988,0.027351,0.043473
BK,0.025343,0.034641,0.043463,0.036350,0.073424,0.032147,0.023762,0.037984,0.043959,0.064969,...,0.020377,0.011479,0.042192,0.075075,0.053836,0.062486,0.041757,0.052466,0.027388,0.044548


In [13]:
min_var_weights: np.ndarray = mvp(cov_matrix.values, annualized_returns.values)
min_var_weights

array([4.47630237e-02, 1.16949177e-01, 6.13251854e-19, 4.15997048e-02,
       3.50053567e-02, 1.70687104e-02, 8.24044955e-02, 6.43214724e-02,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 9.48482531e-03,
       5.45331324e-02, 5.26515680e-18, 4.53954902e-03, 0.00000000e+00,
       0.00000000e+00, 1.37628125e-01, 1.11808349e-17, 7.54536949e-18,
       5.49423458e-02, 3.14660673e-01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 3.36780300e-18, 1.48400172e-18,
       2.20994093e-02, 0.00000000e+00])

In [14]:
results: np.ndarray = efficient_frontier(annualized_returns.values, cov_matrix.values)
results

array([[0.16411927, 0.17984761, 0.16513776, ..., 0.17200057, 0.16029288,
        0.14906666],
       [0.21831385, 0.21580277, 0.21629806, ..., 0.20795144, 0.21844645,
        0.20689464],
       [0.57769706, 0.6573021 , 0.58778961, ..., 0.64438397, 0.55983004,
        0.53682715]])

In [15]:
# TODO: Review that error
plot_efficient_frontier(results)

ValueError: The polynomial coefficient is negative